In [1]:
# install ktrain on Google Colab
!pip3 install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 48.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.41.3-py3-none-any.whl size=25316960 sha256=a03c4807f9a733bab7ea0c6f2c5d73731f758a1d5e8f2cd704b25194e3d81bf6
  Stored in directory: /root/.cache/pip/wheels/fc/76/11/5b953090eebf531f660948a30cd26e70260619f6480f186a5a
  Created wheel for keras_bert: filename=k

In [2]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_train = pd.read_csv('/content/drive/MyDrive/NLP_PROJECT/Train.csv', encoding='utf-8')
data_test = pd.read_csv('/content/drive/MyDrive/NLP_PROJECT/Test.csv', encoding='utf-8')

X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = pd.concat([data_train, data_test], ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'surprise', 'neutral', 'disgust', 'shame', 'guilt']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 31275
size of validation set: 13404
Emotion
joy         12605
sadness      8364
fear         6914
anger        5851
surprise     4275
neutral      2254
disgust      2018
shame        1346
guilt        1052
Name: count, dtype: int64


,Text,Emotion
0,Such as ?,neutral
1,I felt angry when I saw on T.V the last slaugh...,anger
2,Home from a long fun day with my girls @ colle...,joy
3,Apparently everyone at work but me knew it was...,surprise
4,"“@RevRunWisdom: not afraid of tomorrow, for I ...",fear
5,"For Christmas, my parents go shopping together...",surprise
6,When I found out that I could not go to the co...,sadness
7,Even my father was n't too pleased about conti...,joy
8,My grandmother is a highly authoritative perso...,anger
9,@IAM_FET &quot;Truth Is: I find in sharing The...,joy


In [5]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'surprise': 4,
    'neutral': 5,
    'disgust': 6,
    'shame': 7,
    'guilt': 8
}

# Integer labels
# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [6]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350,
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [7]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


done.


In [8]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [9]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
5213/5213 [==============================] - 3060s 582ms/step - loss: 1.1411 - accuracy: 0.6054 - val_loss: 0.8271 - val_accuracy: 0.7150
Epoch 2/3
5213/5213 [==============================] - 3032s 582ms/step - loss: 0.6820 - accuracy: 0.7662 - val_loss: 0.7185 - val_accuracy: 0.7604
Epoch 3/3
5213/5213 [==============================] - 3033s 582ms/step - loss: 0.2916 - accuracy: 0.9048 - val_loss: 0.8037 - val_accuracy: 0.7675


In [10]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

419/419 [==============================] - 368s 868ms/step
              precision    recall  f1-score   support

         joy       0.80      0.82      0.81      3753
     sadness       0.75      0.76      0.75      2541
        fear       0.84      0.82      0.83      2109
       anger       0.77      0.75      0.76      1718
    surprise       0.67      0.66      0.66      1260
     neutral       0.80      0.84      0.81       698
     disgust       0.65      0.58      0.61       593
       shame       0.68      0.67      0.67       403
       guilt       0.68      0.63      0.65       329

    accuracy                           0.77     13404
   macro avg       0.74      0.73      0.73     13404
weighted avg       0.77      0.77      0.77     13404



array([[3094,  201,   87,   49,  207,   68,   27,   19,    1],
       [ 243, 1938,   71,  108,   92,   30,   34,   12,   13],
       [ 110,  100, 1727,   50,   56,   10,   28,   12,   16],
       [  91,  133,   52, 1296,   36,   34,   53,   10,   13],
       [ 220,   95,   56,   27,  826,    4,   29,    3,    0],
       [  47,   34,    7,   25,    1,  583,    1,    0,    0],
       [  37,   54,   33,   77,   19,    1,  345,   23,    4],
       [  12,   19,   10,   30,    0,    1,    8,  270,   53],
       [   6,   26,    5,   25,    1,    2,    6,   50,  208]])

In [11]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy',
 'sadness',
 'fear',
 'anger',
 'surprise',
 'neutral',
 'disgust',
 'shame',
 'guilt']

In [12]:
import time

message = 'I just broke up with my boyfriend'

start_time = time.time()
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: sadness (0.15)


In [13]:
model_path = '/content/drive/MyDrive/NLP_PROJECT/emotion_classifier'
predictor.save(model_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
model_path = '/content/drive/MyDrive/NLP_PROJECT/emotion_classifier'
predictor.save(model_path + '.keras')
